# Find Most Similar Non-Example Pages to Administrative Forms

This notebook uses the Donut model to find pages in the non-examples folder that are most similar to the administrative forms in the examples folder.

## Approach:
1. Extract visual embeddings from the Donut encoder for all example form pages
2. Extract embeddings for all non-example pages
3. Compute similarity scores and find the most similar non-example page
4. Visualize the results

## 1. Setup and Dependencies

In [1]:
import os
# Set tokenizers parallelism to false to avoid fork warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import torch
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from pdf2image import convert_from_path
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from typing import List, Tuple, Dict
import json

from transformers import DonutProcessor, VisionEncoderDecoderModel

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cpu


# Define paths
BASE_PATH = Path('/Users/admin-tascott/Documents/GitHub/chehalis')
EXAMPLE_FORMS_PATH = BASE_PATH / 'data' / 'raw' / '_exampleforms'
NON_EXAMPLES_PATH = BASE_PATH / 'data' / 'raw' / '_nonexamples'

# Model selection - Change to CLIP
MODEL_TYPE = "clip"  # Options: "clip", "donut"

# Load appropriate model based on selection
if MODEL_TYPE == "clip":
    from transformers import CLIPProcessor, CLIPModel
    print("Loading CLIP model")
    model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
    # Use vision model only for embeddings
    model = model.vision_model.to(device)
elif MODEL_TYPE == "donut":
    # Check if trained model exists, otherwise use base model
    MODEL_DIR = "./form_classifier_model"
    if os.path.exists(MODEL_DIR):
        print(f"Loading trained Donut model from {MODEL_DIR}")
        model = VisionEncoderDecoderModel.from_pretrained(MODEL_DIR)
        processor = DonutProcessor.from_pretrained(MODEL_DIR)
    else:
        print("Loading base Donut model")
        MODEL_NAME = "naver-clova-ix/donut-base"
        processor = DonutProcessor.from_pretrained(MODEL_NAME)
        model = VisionEncoderDecoderModel.from_pretrained(MODEL_NAME)
    # Use encoder only for embeddings
    model = model.encoder.to(device)

model.eval()  # Set to evaluation mode
print(f"Model loaded successfully: {MODEL_TYPE}")

In [ ]:
def pdf_to_images(pdf_path: Path, dpi: int = 200) -> List[Image.Image]:
    """Convert PDF to list of PIL Images"""
    try:
        images = convert_from_path(pdf_path, dpi=dpi)
        return images
    except Exception as e:
        print(f"Error converting {pdf_path}: {e}")
        return []

def preprocess_image_for_model(image: Image.Image, model_type: str = "clip") -> Image.Image:
    """Preprocess image based on model type"""
    if image.mode != 'RGB':
        image = image.convert('RGB')
    
    if model_type == "clip":
        # CLIP expects 224x224 images
        image.thumbnail((224, 224), Image.Resampling.LANCZOS)
    else:  # donut
        # Donut expects larger images
        image.thumbnail((1280, 960), Image.Resampling.LANCZOS)
    
    return image

def extract_embeddings(image: Image.Image, model, processor, device, model_type: str = "clip"):
    """
    Extract visual embeddings from model
    
    Returns:
        numpy array of embeddings
    """
    # Preprocess image
    image = preprocess_image_for_model(image, model_type)
    
    # Get pixel values
    inputs = processor(images=image, return_tensors="pt")
    pixel_values = inputs.pixel_values.to(device)
    
    with torch.no_grad():
        if model_type == "clip":
            # For CLIP, we already have the vision model
            outputs = model(pixel_values=pixel_values)
        else:  # donut
            # For Donut encoder
            outputs = model(pixel_values)
        
        # Extract embeddings
        if hasattr(outputs, 'pooler_output') and outputs.pooler_output is not None:
            embeddings = outputs.pooler_output
        elif hasattr(outputs, 'last_hidden_state'):
            # Mean pool the last hidden states
            embeddings = outputs.last_hidden_state.mean(dim=1)
        else:
            # For some models, outputs might be a tuple
            embeddings = outputs[0].mean(dim=1) if isinstance(outputs, tuple) else outputs
        
        embeddings = embeddings.cpu().numpy()
    
    return embeddings[0]  # Return first (and only) embedding in batch

In [ ]:
# Extract embeddings for all example form pages
example_embeddings = []
example_metadata = []

print("Processing example forms...")
pdf_files = list(EXAMPLE_FORMS_PATH.glob('*.pdf'))

for pdf_path in tqdm(pdf_files, desc="Example PDFs"):
    images = pdf_to_images(pdf_path)
    for page_num, image in enumerate(images):
        embedding = extract_embeddings(image, model, processor, device, MODEL_TYPE)
        example_embeddings.append(embedding)
        example_metadata.append({
            'file_path': str(pdf_path),
            'filename': pdf_path.name,
            'page_num': page_num + 1,
            'total_pages': len(images)
        })

example_embeddings = np.array(example_embeddings)
print(f"\nExtracted embeddings for {len(example_embeddings)} example form pages")
print(f"Embedding shape: {example_embeddings.shape}")

# Extract embeddings for all non-example pages
non_example_embeddings = []
non_example_metadata = []
non_example_images = []  # Store for visualization

print("\nProcessing non-example documents...")
pdf_files = list(NON_EXAMPLES_PATH.glob('*.pdf'))

# Limit number of PDFs for memory management (adjust as needed)
max_pdfs = 50  # Process first 50 PDFs
pdf_files = pdf_files[:max_pdfs]

for pdf_path in tqdm(pdf_files, desc="Non-example PDFs"):
    images = pdf_to_images(pdf_path)
    for page_num, image in enumerate(images):
        embedding = extract_embeddings(image, model, processor, device, MODEL_TYPE)
        non_example_embeddings.append(embedding)
        non_example_metadata.append({
            'file_path': str(pdf_path),
            'filename': pdf_path.name,
            'page_num': page_num + 1,
            'total_pages': len(images)
        })
        # Store resized image for visualization
        non_example_images.append(preprocess_image_for_model(image, MODEL_TYPE))

non_example_embeddings = np.array(non_example_embeddings)
print(f"\nExtracted embeddings for {len(non_example_embeddings)} non-example pages")
print(f"Embedding shape: {non_example_embeddings.shape}")

In [5]:
def extract_embeddings(image: Image.Image, model, processor, device):
    """
    Extract visual embeddings from Donut encoder
    
    Returns:
        numpy array of embeddings
    """
    # Preprocess image
    image = preprocess_image_for_donut(image)
    
    # Get pixel values
    pixel_values = processor(image, return_tensors="pt").pixel_values.to(device)
    
    with torch.no_grad():
        # Get encoder outputs
        encoder_outputs = model.encoder(pixel_values)
        
        # Use the pooled output or mean of last hidden states
        if hasattr(encoder_outputs, 'pooler_output') and encoder_outputs.pooler_output is not None:
            embeddings = encoder_outputs.pooler_output
        else:
            # Mean pool the last hidden states
            embeddings = encoder_outputs.last_hidden_state.mean(dim=1)
        
        embeddings = embeddings.cpu().numpy()
    
    return embeddings[0]  # Return first (and only) embedding in batch

## 5. Process All Example Forms

In [ ]:
# Visualize top 5 most similar non-example pages
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
fig.suptitle('Top 5 Most Similar Non-Example Pages', fontsize=16)

for i, idx in enumerate(top_k_indices[:5]):
    # Non-example page
    ax_non = axes[0, i]
    ax_non.imshow(non_example_images[idx])
    ax_non.set_title(f"Non-Example\n{non_example_metadata[idx]['filename']}\nPage {non_example_metadata[idx]['page_num']}\nScore: {max_similarities[idx]:.3f}")
    ax_non.axis('off')
    
    # Most similar example page
    similar_example_idx = most_similar_example_idx[idx]
    example_meta = example_metadata[similar_example_idx]
    
    # Load the example image for visualization
    example_pdf_path = Path(example_meta['file_path'])
    example_images = pdf_to_images(example_pdf_path)
    example_image = preprocess_image_for_model(example_images[example_meta['page_num'] - 1], MODEL_TYPE)
    
    ax_ex = axes[1, i]
    ax_ex.imshow(example_image)
    ax_ex.set_title(f"Similar Example\n{example_meta['filename']}\nPage {example_meta['page_num']}")
    ax_ex.axis('off')

plt.tight_layout()
plt.show()

## 6. Process All Non-Example Pages

In [10]:
# Extract embeddings for all non-example pages
non_example_embeddings = []
non_example_metadata = []
non_example_images = []  # Store for visualization

print("\nProcessing non-example documents...")
pdf_files = list(NON_EXAMPLES_PATH.glob('*.pdf'))

# Limit number of PDFs for memory management (adjust as needed)
max_pdfs = 50  # Process first 50 PDFs
pdf_files = pdf_files[:max_pdfs]

for pdf_path in tqdm(pdf_files, desc="Non-example PDFs"):
    images = pdf_to_images(pdf_path)
    for page_num, image in enumerate(images):
        embedding = extract_embeddings(image, model, processor, device)
        non_example_embeddings.append(embedding)
        non_example_metadata.append({
            'file_path': str(pdf_path),
            'filename': pdf_path.name,
            'page_num': page_num + 1,
            'total_pages': len(images)
        })
        # Store resized image for visualization
        non_example_images.append(preprocess_image_for_donut(image))

non_example_embeddings = np.array(non_example_embeddings)
print(f"\nExtracted embeddings for {len(non_example_embeddings)} non-example pages")
print(f"Embedding shape: {non_example_embeddings.shape}")


Processing non-example documents...


Non-example PDFs: 100%|██████████████████████| 50/50 [1:36:16<00:00, 115.53s/it]


Extracted embeddings for 1073 non-example pages
Embedding shape: (1073, 1024)


## 7. Compute Similarity Scores

In [15]:
# Compute cosine similarity between all example and non-example pages
print("\nComputing similarity scores...")
similarity_matrix = cosine_similarity(non_example_embeddings, example_embeddings)

# Find the maximum similarity for each non-example page
max_similarities = similarity_matrix.max(axis=1)
most_similar_example_idx = similarity_matrix.argmax(axis=1)

# Find the overall most similar non-example page
most_similar_idx = max_similarities.argmax()
most_similar_score = max_similarities[most_similar_idx]
most_similar_example = most_similar_example_idx[most_similar_idx]

print(f"\nMost similar non-example page:")
print(f"  File: {non_example_metadata[most_similar_idx]['filename']}")
print(f"  Page: {non_example_metadata[most_similar_idx]['page_num']}")
print(f"  Similarity score: {most_similar_score:.4f}")
print(f"  Most similar to example: {example_metadata[most_similar_example]['filename']}, page {example_metadata[most_similar_example]['page_num']}")


Computing similarity scores...

Most similar non-example page:
  File: 739-000.pdf
  Page: 15
  Similarity score: 0.9985
  Most similar to example: 747-000.pdf, page 2


## 8. Find Top K Most Similar Pages

In [23]:
# Get top K most similar non-example pages
K = 10
top_k_indices = np.argsort(max_similarities)[-K:][::-1]

print(f"\nTop {K} most similar non-example pages:")
results = []

for rank, idx in enumerate(top_k_indices):
    metadata = non_example_metadata[idx]
    similar_example_idx = most_similar_example_idx[idx]
    similar_example = example_metadata[similar_example_idx]
    
    result = {
        'rank': rank + 1,
        'filename': metadata['filename'],
        'page': metadata['page_num'],
        'similarity_score': max_similarities[idx],
        'similar_to_example': similar_example['filename'],
        'similar_to_page': similar_example['page_num']
    }
    results.append(result)
    
    print(f"\n{rank + 1}. File: {metadata['filename']}, Page: {metadata['page_num']}")
    print(f"   Similarity: {max_similarities[idx]:.4f}")
    print(f"   Most similar to: {similar_example['filename']}, page {similar_example['page_num']}")

# Create DataFrame for easy viewing
results_df = pd.DataFrame(results)
results_df


Top 10 most similar non-example pages:

1. File: 739-000.pdf, Page: 15
   Similarity: 0.9985
   Most similar to: 747-000.pdf, page 2

2. File: 1197-000.pdf, Page: 1
   Similarity: 0.9936
   Most similar to: 739-000.pdf, page 1

3. File: 25581-000.pdf, Page: 20
   Similarity: 0.9933
   Most similar to: 1050-002.pdf, page 1

4. File: 1197-000.pdf, Page: 5
   Similarity: 0.9928
   Most similar to: 730-000.pdf, page 1

5. File: 25581-000.pdf, Page: 21
   Similarity: 0.9923
   Most similar to: 719-000.pdf, page 1

6. File: 1197-000.pdf, Page: 4
   Similarity: 0.9920
   Most similar to: 719-000.pdf, page 1

7. File: 104473-000.pdf, Page: 34
   Similarity: 0.9919
   Most similar to: 9697-000.pdf, page 1

8. File: 25581-000.pdf, Page: 11
   Similarity: 0.9916
   Most similar to: 751-000.pdf, page 1

9. File: 732-001.pdf, Page: 14
   Similarity: 0.9910
   Most similar to: 1064-000.pdf, page 1

10. File: F1-9-DCS-43810SAS-001.pdf, Page: 4
   Similarity: 0.9909
   Most similar to: 1021-000.pdf, 

,rank,filename,page,similarity_score,similar_to_example,similar_to_page
0,1,739-000.pdf,15,0.998489,747-000.pdf,2
1,2,1197-000.pdf,1,0.993603,739-000.pdf,1
2,3,25581-000.pdf,20,0.993346,1050-002.pdf,1
3,4,1197-000.pdf,5,0.992799,730-000.pdf,1
4,5,25581-000.pdf,21,0.992268,719-000.pdf,1
5,6,1197-000.pdf,4,0.992050,719-000.pdf,1
6,7,104473-000.pdf,34,0.991850,9697-000.pdf,1
7,8,25581-000.pdf,11,0.991611,751-000.pdf,1
8,9,732-001.pdf,14,0.990955,1064-000.pdf,1
9,10,F1-9-DCS-43810SAS-001.pdf,4,0.990906,1021-000.pdf,1


## 9. Visualize Most Similar Pages

In [ ]:
# Visualize top 5 most similar non-example pages
fig, axes = plt.subplots(2, 5, figsize=(20, 10))
fig.suptitle('Top 5 Most Similar Non-Example Pages', fontsize=16)

for i, idx in enumerate(top_k_indices[:5]):
    # Non-example page
    ax_non = axes[0, i]
    ax_non.imshow(non_example_images[idx])
    ax_non.set_title(f"Non-Example\n{non_example_metadata[idx]['filename']}\nPage {non_example_metadata[idx]['page_num']}\nScore: {max_similarities[idx]:.3f}")
    ax_non.axis('off')
    
    # Most similar example page
    similar_example_idx = most_similar_example_idx[idx]
    example_meta = example_metadata[similar_example_idx]
    
    # Load the example image for visualization
    example_pdf_path = Path(example_meta['file_path'])
    example_images = pdf_to_images(example_pdf_path)
    example_image = preprocess_image_for_donut(example_images[example_meta['page_num'] - 1])
    
    ax_ex = axes[1, i]
    ax_ex.imshow(example_image)
    ax_ex.set_title(f"Similar Example\n{example_meta['filename']}\nPage {example_meta['page_num']}")
    ax_ex.axis('off')

plt.tight_layout()
plt.show()

## 10. Analyze Similarity Distribution

In [ ]:
# Plot distribution of similarity scores
plt.figure(figsize=(10, 6))
plt.hist(max_similarities, bins=50, alpha=0.7, edgecolor='black')
plt.axvline(x=most_similar_score, color='red', linestyle='--', label=f'Most similar: {most_similar_score:.3f}')
plt.xlabel('Maximum Cosine Similarity Score')
plt.ylabel('Number of Non-Example Pages')
plt.title('Distribution of Similarity Scores\n(Non-Example Pages vs Most Similar Example Form)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Statistics
print(f"\nSimilarity Statistics:")
print(f"Mean similarity: {max_similarities.mean():.4f}")
print(f"Std deviation: {max_similarities.std():.4f}")
print(f"Min similarity: {max_similarities.min():.4f}")
print(f"Max similarity: {max_similarities.max():.4f}")
print(f"\nPages with similarity > 0.9: {(max_similarities > 0.9).sum()}")
print(f"Pages with similarity > 0.8: {(max_similarities > 0.8).sum()}")
print(f"Pages with similarity > 0.7: {(max_similarities > 0.7).sum()}")

## 11. Save Results

In [ ]:
# Save detailed results to CSV
all_results = []

for idx, (score, example_idx) in enumerate(zip(max_similarities, most_similar_example_idx)):
    metadata = non_example_metadata[idx]
    similar_example = example_metadata[example_idx]
    
    all_results.append({
        'non_example_file': metadata['filename'],
        'non_example_page': metadata['page_num'],
        'non_example_total_pages': metadata['total_pages'],
        'similarity_score': score,
        'most_similar_example_file': similar_example['filename'],
        'most_similar_example_page': similar_example['page_num']
    })

results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('similarity_score', ascending=False)
results_df.to_csv('similarity_results.csv', index=False)

print(f"\nResults saved to similarity_results.csv")
print(f"\nTop 10 most similar pages:")
print(results_df.head(10))

## 12. Identify Potential Misclassified Pages

In [ ]:
# Pages with very high similarity might be misclassified
threshold = 0.85  # Adjust based on your needs
potential_misclassified = results_df[results_df['similarity_score'] > threshold]

print(f"\nPotential misclassified pages (similarity > {threshold}):")
print(f"Found {len(potential_misclassified)} pages")

if len(potential_misclassified) > 0:
    print("\nThese non-example pages are very similar to the administrative forms:")
    for _, row in potential_misclassified.iterrows():
        print(f"\n- File: {row['non_example_file']}, Page: {row['non_example_page']}")
        print(f"  Similarity: {row['similarity_score']:.4f}")
        print(f"  Similar to: {row['most_similar_example_file']}, page {row['most_similar_example_page']}")
    
    # Save potential misclassified pages
    potential_misclassified.to_csv('potential_misclassified_pages.csv', index=False)
    print("\nSaved to potential_misclassified_pages.csv")

## Usage Notes

This notebook helps identify:

1. **Most Similar Pages**: Which pages in the non-examples are most visually similar to the administrative forms
2. **Potential Misclassifications**: Pages with very high similarity scores might actually be administrative forms that were incorrectly placed in the non-examples folder
3. **Similarity Distribution**: Understanding how similar non-form pages are to form pages helps set classification thresholds

### Next Steps:
- Review pages with high similarity scores (>0.85) to check if they're actually administrative forms
- Use the similarity threshold to improve the classifier's decision boundary
- Consider adding highly similar non-forms as hard negative examples in training